In [ ]:
import pandas as pd
import numpy as np
import importlib
import tTsTGrpUtils as tsutil

In [ ]:
# load in all dls up to now
importlib.reload(tsutil)
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
pth_04a_maps = "04a_dl_maps_08Oct2025-104443.pkl"
pth_04a_parc = "04b_dl_maps_parcel_08Oct2025-110220.pkl"
pth_05a_winStudy = "05a_winStudy_08Oct2025-110338.pkl"
pth_05b_winStudy_ic = "05b_stats_winStudy_grp_08Oct2025-110654.pkl"

dl_maps = tsutil.loadPickle(root + pth_04a_maps, verbose = True)
dl_parc = tsutil.loadPickle(root + pth_04a_parc, verbose = True)
dl_winComp = tsutil.loadPickle(root + pth_05a_winStudy, verbose = True)
dl_grp_ic = tsutil.loadPickle(root + pth_05b_winStudy_ic, verbose = True)

In [ ]:
index = 12
dl_maps_test = dl_maps[index]
txt = tsutil.printItemMetadata(dl_maps_test, return_txt=True)
print(f"dl_maps: {txt}")

dl_parc_test = dl_parc[index]
txt = tsutil.printItemMetadata(dl_parc_test, return_txt=True)
print(f"dl_maps: {txt}")

dl_winComp_test = dl_winComp[index]
txt = tsutil.printItemMetadata(dl_winComp_test, return_txt=True)
print(f"dl_winComp: {txt}")

dl_grp_ic_test = dl_grp_ic[index]
txt = tsutil.printItemMetadata(dl_grp_ic_test, return_txt=True)
print(f"dl_grp_ic: {txt}")

In [ ]:
# print keys for each test item
print(f"dl_maps keys:\n\t{list(dl_maps_test.keys())}")
print(f"dl_parc keys:\n\t{list(dl_parc_test.keys())}")
print(f"dl_winComp keys:\n\t{list(dl_winComp_test.keys())}")
print(f"dl_grp_ic keys:\n\t{list(dl_grp_ic_test.keys())}")

In [ ]:
if dl_maps_test['region'] == 'cortex':
    dl_maps_key = 'df_maps'
    dl_parc_key = 'df_maps_parc_glsr_mdn'
    dl_winComp_key = 'df_maps_parc_glsr_mdn_z'
    dl_grp_ic_key = 'df_maps_parc_glsr_mdn_z_TLE_ic'
else:
    dl_maps_key = 'df_maps'
    dl_parc_key = 'df_maps_parc_dk25_mdn'
    dl_winComp_key = 'df_maps_parc_dk25_mdn_z'
    dl_grp_ic_key = 'df_maps_parc_dk25_mdn_z_TLE_ic'

# print indices and col names for each
df_maps = dl_maps_test[dl_maps_key]
if type(df_maps) is str:
    df_maps = tsutil.loadPickle(df_maps, verbose = True)
print(f"dl_maps <{df_maps.shape}>\n\t{list(df_maps.columns)}")

df_parc = dl_parc_test[dl_parc_key]
if type(df_parc) is str:
    df_parc = tsutil.loadPickle(df_parc, verbose = True)
print(f"dl_parc <{df_parc.shape}>\n\t{list(df_parc.columns)}")

df_winComp = dl_winComp_test[dl_winComp_key]
if type(df_winComp) is str:
    df_winComp = tsutil.loadPickle(df_winComp, verbose = True)
print(f"dl_winComp <{df_winComp.shape}>\n\t{list(df_winComp.columns)}")

df_grp_ic = dl_grp_ic_test[dl_grp_ic_key]
if type(df_grp_ic) is str:
    df_grp_ic = tsutil.loadPickle(df_grp_ic, verbose = True)
print(f"dl_grp_ic <{df_grp_ic.shape}>\n\t{list(df_grp_ic.columns)}")

In [ ]:
importlib.reload(tsutil)
df_parc = tsutil.apply_glasser(df_maps, surf = dl_maps_test['surf'])
df_parc = df_parc.groupby(df_parc.columns, axis=1).median()
df_parc.columns
df_parc.shape